In [15]:
%load_ext autoreload
%autoreload 2

In [44]:
import pandas as pd
import re

from utils import get_data, Config

In [57]:
# Load all data
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Response".id <= 116032;
"""
df = get_data(Config(), initial_data)

Connecting to the PostgreSQL database...
Connection successful


In [104]:
# Extra columns
df["outcome_str"] = df["outcome_value"].fillna("None").astype(str)

## Overview

In [60]:
display(df.head())
display(df.columns)

,id,outcome_type,outcome_value,test_name,test_status,test_message,test_stack,org_scheme,org_host,resp_scheme,...,raw_header,status_code,label,resp_type,name,version,headless_mode,os,automation_mode,add_info
0,1,<class 'dict'>,"{'error': 'object ""TypeError: Failed to fetch""...",fetch_GET,0,None,None,https,sub.headers.websec.saarland,http,...,"[[Test, Test], [Access-Control-Allow-Orgin, *]...",200,CORS,basic,chrome,119,headless-new,Ubuntu 22.04,selenium,
1,2,<class 'dict'>,"{'error': 'object ""TypeError: Failed to fetch""...",fetch_TEST,0,None,None,https,sub.headers.websec.saarland,http,...,"[[Test, Test], [Access-Control-Allow-Orgin, *]...",200,CORS,basic,chrome,119,headless-new,Ubuntu 22.04,selenium,
2,3,<class 'dict'>,"{'error': 'object ""TypeError: Failed to fetch""...",fetch_GET,0,None,None,https,sub.headers.websec.saarland,http,...,"[[Test, Test], [Access-Control-Allow-Orgin, *]...",200,CORS,basic,chrome,119,headless-new,Ubuntu 22.04,selenium,
3,4,<class 'dict'>,"{'error': 'object ""TypeError: Failed to fetch""...",fetch_GET,0,None,None,https,sub.headers.websec.saarland,http,...,"[[Test, Test], [Access-Control-Allow-Orgin, *]...",200,CORS,basic,chrome,119,headless-new,Ubuntu 22.04,selenium,
4,5,<class 'dict'>,"{'error': 'null', 'headers': 'access-control-a...",fetch_GET,0,None,None,https,sub.headers.websec.saarland,https,...,"[[Test, Test], [Access-Control-Allow-Orgin, *]...",200,CORS,basic,chrome,119,headless-new,Ubuntu 22.04,selenium,


Index(['id', 'outcome_type', 'outcome_value', 'test_name', 'test_status',
       'test_message', 'test_stack', 'org_scheme', 'org_host', 'resp_scheme',
       'resp_host', 'relation_info', 'browser_id', 'testcase_id',
       'response_id', 'status', 'created_at', 'updated_at', 'raw_header',
       'status_code', 'label', 'resp_type', 'name', 'version', 'headless_mode',
       'os', 'automation_mode', 'add_info'],
      dtype='object')

In [71]:
# Collected results per browser
df.groupby(["name", "version", "os", "headless_mode", "automation_mode"])["id"].count().sort_values(ascending=False).to_frame()

id
name    version                   os           headless_mode automation_mode       
brave   1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium         11624
                                               xvfb          selenium         11624
        1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium         11624
                                               xvfb          selenium         11624
chrome  119                       Ubuntu 22.04 headless-new  selenium         11624
                                               xvfb          selenium         11624
edge    119                       Ubuntu 22.04 headless-new  selenium         11624
                                               xvfb          selenium         11624
firefox 119                       Ubuntu 22.04 xvfb          selenium         11528
                                               headless      selenium         11512

In [74]:
# Timeouts
display(df.groupby(["name", "version", "os", "headless_mode", "automation_mode"])["test_status"].value_counts().to_frame())

display(df.loc[df["test_status"] == 2])

count
name    version                   os           headless_mode automation_mode test_status       
brave   1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium        0            11624
                                               xvfb          selenium        0            11624
        1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium        0            11624
                                               xvfb          selenium        0            11624
chrome  119                       Ubuntu 22.04 headless-new  selenium        0            11624
                                               xvfb          selenium        0            11624
edge    119                       Ubuntu 22.04 headless-new  selenium        0            11624
                                               xvfb          selenium        0            11622
                                                                             2                2
firefox 119                       Ubuntu 22.04 headless      selenium        0            11512
                                               xvfb          selenium        0            11527
                                                                             2                1

,id,outcome_type,outcome_value,test_name,test_status,test_message,test_stack,org_scheme,org_host,resp_scheme,...,raw_header,status_code,label,resp_type,name,version,headless_mode,os,automation_mode,add_info
73595,73631,<class 'NoneType'>,None,referrer_iframe,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,"[[Referrer-Policy, no-referrer]]",200,RP,basic,firefox,119,xvfb,Ubuntu 22.04,selenium,
84995,84976,<class 'NoneType'>,None,oac_window.open,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,"[[origin-agent-cluster, ?0], [location, https:...",302,OAC,basic,edge,119,xvfb,Ubuntu 22.04,selenium,
85333,85352,<class 'NoneType'>,None,referrer_iframe,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,"[[Referrer-Policy, unsafe-url], [location, htt...",302,RP,basic,edge,119,xvfb,Ubuntu 22.04,selenium,


In [80]:
# Time taken!
# Currently the HTTP times of Chromium-based browsers do not make too much sense due to HTTPS upgrades; however this is fixed for the next run
df.groupby(["name", "version", "os", "headless_mode", "automation_mode", "org_scheme"])["created_at"].agg(["min", "max"]).apply(lambda x: x["max"] - x["min"], axis=1).to_frame()

0
name    version                   os           headless_mode automation_mode org_scheme                       
brave   1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium        http       0 days 00:16:54.759901
                                                                             https      0 days 00:32:53.405975
                                               xvfb          selenium        http       0 days 00:16:57.517750
                                                                             https      0 days 00:32:59.348796
        1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium        http       0 days 00:16:53.083071
                                                                             https      0 days 00:32:52.104873
                                               xvfb          selenium        http       0 days 00:16:58.926585
                                                                             https      0 days 00:33:05.305826
chrome  119                       Ubuntu 22.04 headless-new  selenium        http       0 days 00:00:49.338957
                                                                             https      0 days 00:32:25.278693
                                               xvfb          selenium        http       0 days 00:00:49.509877
                                                                             https      0 days 00:32:28.764379
edge    119                       Ubuntu 22.04 headless-new  selenium        http       0 days 00:12:11.420509
                                                                             https      0 days 00:16:02.139976
                                               xvfb          selenium        http       0 days 00:12:15.619910
                                                                             https      0 days 00:16:03.400237
firefox 119                       Ubuntu 22.04 headless      selenium        http       0 days 00:14:08.454610
                                                                             https      0 days 00:17:07.358566
                                               xvfb          selenium        http       0 days 00:15:20.420077
                                                                             https      0 days 00:18:06.464448

## Response analysis

- We have 11624 unique tests: (test_name x relation_info) x org_scheme x (resp_scheme x resp_host) x response_id)
- They should all have 10 (num browsers in various settings) outcomes
    -  Due to the HTTPS Upgrade bug many have either 16 or 4 outcomes only (as the 6 http outcomes of Chrome + Brave instead resulted in HTTPS outcomes)
    -  Without org_scheme they should have 20; a couple tests have less than 20 (due to the Firefox errors and/or timeouts?)

In [142]:
# Unique tests
tests = {
    "Fetch": [4, 2, 8, 6], 
    "Framing": [9, 2, 8, 35], # 7+11+17
    "PP": [4, 2, 8, 24],
    "OAC": [3, 2, 8, 14],
    "TAO": [1, 2, 8, 13],
    "RP": [2, 2, 8, 16],
    "CSP-SCRIPT": [2, 2, 8, 17],
    "COEP": [2, 2, 8, 11],
    "CORP": [4, 2, 8, 11],
    "CSP-IMG": [1, 2, 8, 17],
    "HSTS": [2, 1, 2, 14],
    "COOP": [1, 2, 8, 11],
}
tests = pd.DataFrame.from_dict(tests, columns=["tests", "org_origins", "resp_origins", "responses"], orient="index")
tests["All tests"] = tests.prod(axis=1)
tests.loc["Sum"] = tests.sum(axis=0)
tests

,tests,org_origins,resp_origins,responses,All tests
Fetch,4,2,8,6,384
Framing,9,2,8,35,5040
PP,4,2,8,24,1536
OAC,3,2,8,14,672
TAO,1,2,8,13,208
RP,2,2,8,16,512
CSP-SCRIPT,2,2,8,17,544
COEP,2,2,8,11,352
CORP,4,2,8,11,704
CSP-IMG,1,2,8,17,272


In [134]:
# SELECT count(*) as c2, count(DISTINCT browser_id), count(DISTINCT outcome_value) as c1, test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id 
# FROM "Result" GROUP by test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id order by c1 DESC, c2 DESC

# Currently ignore "org_scheme"
res = df.loc[df["org_scheme"] == "https"].groupby(["test_name", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])[["browser_id", "outcome_str"]].agg(["count", "nunique"])
display(res)
display(res.value_counts().to_frame())

browser_id  \
                                                                                                                                     count   
test_name                     org_host                    resp_scheme resp_host                       relation_info response_id              
accesswindow_direct           sub.headers.websec.saarland http        headers.webappsec.eu            direct        1524                16   
                                                                                                                    1525                16   
                                                                                                                    1526                16   
                                                                                                                    1527                16   
                                                                                                                    1528                16   
...                                                                                                                                    ...   
subresourceloadingCORP_object sub.headers.websec.saarland https       sub.sub.headers.websec.saarland direct        1907                16   
                                                                                                                    2113                16   
                                                                                                                    2114                16   
                                                                                                                    2115                16   
                                                                                                                    2116                16   

                                                                                                                                         \
                                                                                                                                nunique   
test_name                     org_host                    resp_scheme resp_host                       relation_info response_id           
accesswindow_direct           sub.headers.websec.saarland http        headers.webappsec.eu            direct        1524             10   
                                                                                                                    1525             10   
                                                                                                                    1526             10   
                                                                                                                    1527             10   
                                                                                                                    1528             10   
...                                                                                                                                 ...   
subresourceloadingCORP_object sub.headers.websec.saarland https       sub.sub.headers.websec.saarland direct        1907             10   
                                                                                                                    2113             10   
                                                                                                                    2114             10   
                                                                                                                    2115             10   
                                                                                                                    2116             10   

                                                                                                                                outcome_str  \
                                                 

count
(browser_id, count) (browser_id, nunique) (outcome_str, count) (outcome_str, nunique)       
16                  10                    16                   1                        5018
                                                               2                         452
                                                               3                         138
12                  10                    12                   2                          84
14                  8                     14                   1                          80
12                  10                    12                   1                          12

### Differences (currently only look at HTTPS):
- COOP (accesswindow_direct):
    - Edge + Firefox: redirect first
    - Brave + Chrome: COOP first?
- Fetch (fetch_GET, ...):
    - Error message is different for Chromium-based and Firefox, but same outcome?
-  Framing:
    -  framing_iframe_sandbox: CSP-FA *; Firefox allows, all other browsers deny?
- CORP object/img
- COEP error behavior
- OAC, PP not supported in Firefox (OAC + random behavior)
- Referrer Policy: differen default in Brave
- ...

In [135]:
test_name_seen = set()
for test_name, org_host, resp_scheme, resp_host, relation_info, response_id in res.loc[res[("outcome_str", "nunique")] != 1].index:
    # Only look at each feature group once
    if test_name in test_name_seen:
        continue
    test_name_seen.add(test_name)
    print(test_name)
    # no org_scheme at the moment
    rows = df.loc[df["org_scheme"] == "https"].loc[(df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"] == response_id)]
    with pd.option_context("display.max_colwidth", 200):
        display(rows[["outcome_str", "name"]].value_counts().to_frame())
        display(rows.drop_duplicates(subset="outcome_str")[["browser_id", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info"]])
    

accesswindow_direct


count
outcome_str                                        name          
{'window.open.opener': 'null'}                     brave        8
                                                   chrome       4
{'window.open.opener': 'object "[object Window]"'} edge         2
                                                   firefox      2

,browser_id,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info
5794,13,{'window.open.opener': 'null'},"[[Cross-Origin-Opener-Policy, same-origin-allow-popups], [location, https://sub.headers.websec.saarland/_hp/common/empty.html]]",accesswindow_direct,https,sub.headers.websec.saarland,http,direct
23100,14,"{'window.open.opener': 'object ""[object Window]""'}","[[Cross-Origin-Opener-Policy, same-origin-allow-popups], [location, https://sub.headers.websec.saarland/_hp/common/empty.html]]",accesswindow_direct,https,sub.headers.websec.saarland,http,direct


fetch_GET


count
outcome_str                                                                                     name          
{'error': 'object "TypeError: Failed to fetch"', 'headers': ''}                                 brave        4
                                                                                                chrome       4
                                                                                                edge         2
{'error': 'object "TypeError: NetworkError when attempting to fetch resource."', 'headers': ''} firefox      2

...

count
outcome_str name          
load        brave        8
            chrome       4
error       firefox      2
load        edge         2

,browser_id,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info
5262,13,load,"[[Cross-Origin-Resource-Policy, same-origin]]",subresourceloadingCORP_img,https,sub.headers.websec.saarland,http,direct
22587,14,error,"[[Cross-Origin-Resource-Policy, same-origin]]",subresourceloadingCORP_img,https,sub.headers.websec.saarland,http,direct


subresourceloadingCORP_object


count
outcome_str name          
load        brave        8
            chrome       4
error       firefox      2
load        edge         2

,browser_id,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info
5339,13,load,"[[Cross-Origin-Resource-Policy, same-origin], [location, https://sub.headers.websec.saarland/_hp/common/empty.html]]",subresourceloadingCORP_object,https,sub.headers.websec.saarland,https,direct
22716,14,error,"[[Cross-Origin-Resource-Policy, same-origin], [location, https://sub.headers.websec.saarland/_hp/common/empty.html]]",subresourceloadingCORP_object,https,sub.headers.websec.saarland,https,direct


In [98]:
df.loc[(df["test_name"] == "accesswindow_direct") & (df["org_scheme"] == "https") & (df["resp_scheme"] == "http") & (df["resp_host"] == "headers.webappsec.eu") & (df["relation_info"] == "direct") & (df["response_id"] == 1524)]

,id,outcome_type,outcome_value,test_name,test_status,test_message,test_stack,org_scheme,org_host,resp_scheme,...,status_code,label,resp_type,name,version,headless_mode,os,automation_mode,add_info,outcome_str
5638,5623,<class 'dict'>,"{'window.open.opener': 'object ""[object Window...",accesswindow_direct,0,None,None,https,sub.headers.websec.saarland,http,...,200,COOP,basic,chrome,119,headless-new,Ubuntu 22.04,selenium,,"{'window.open.opener': 'object ""[object Window..."
11580,11463,<class 'dict'>,"{'window.open.opener': 'object ""[object Window...",accesswindow_direct,0,None,None,https,sub.headers.websec.saarland,http,...,200,COOP,basic,chrome,119,headless-new,Ubuntu 22.04,selenium,,"{'window.open.opener': 'object ""[object Window..."
23076,22975,<class 'dict'>,"{'window.open.opener': 'object ""[object Window...",accesswindow_direct,0,None,None,https,sub.headers.websec.saarland,http,...,200,COOP,basic,firefox,119,headless,Ubuntu 22.04,selenium,,"{'window.open.opener': 'object ""[object Window..."
34681,34599,<class 'dict'>,"{'window.open.opener': 'object ""[object Window...",accesswindow_direct,0,None,None,https,sub.headers.websec.saarland,http,...,200,COOP,basic,edge,119,headless-new,Ubuntu 22.04,selenium,,"{'window.open.opener': 'object ""[object Window..."
40430,40383,<class 'dict'>,"{'window.open.opener': 'object ""[object Window...",accesswindow_direct,0,None,None,https,sub.headers.websec.saarland,http,...,200,COOP,basic,brave,1.59.120 (118.0.5993.88),headless-new,Ubuntu 22.04,selenium,,"{'window.open.opener': 'object ""[object Window..."
46216,46223,<class 'dict'>,"{'window.open.opener': 'object ""[object Window...",accesswindow_direct,0,None,None,https,sub.headers.websec.saarland,http,...,200,COOP,basic,brave,1.59.120 (118.0.5993.88),headless-new,Ubuntu 22.04,selenium,,"{'window.open.opener': 'object ""[object Window..."
51960,52007,<class 'dict'>,"{'window.open.opener': 'object ""[object Window...",accesswindow_direct,0,None,None,https,sub.headers.websec.saarland,http,...,200,COOP,basic,brave,1.60.114 (119.0.6045.124),headless-new,Ubuntu 22.04,selenium,,"{'window.open.opener': 'object ""[object Window..."
57793,57847,<class 'dict'>,"{'window.open.opener': 'object ""[object Window...",accesswindow_direct,0,None,None,https,sub.headers.websec.saarland,http,...,200,COOP,basic,brave,1.60.114 (119.0.6045.124),headless-new,Ubuntu 22.04,selenium,,"{'window.open.opener': 'object ""[object Window..."
63562,63631,<class 'dict'>,"{'window.open.opener': 'object ""[object Window...",accesswindow_direct,0,None,None,https,sub.headers.websec.saarland,http,...,200,COOP,basic,chrome,119,xvfb,Ubuntu 22.04,selenium,,"{'window.open.opener': 'object ""[object Window..."
69549,69471,<class 'dict'>,"{'window.open.opener': 'object ""[object Window...",accesswindow_direct,0,None,None,https,sub.headers.websec.saarland,http,...,200,COOP,basic,chrome,119,xvfb,Ubuntu 22.04,selenium,,"{'window.open.opener': 'object ""[object Window..."


# Crash analysis
- Exceptions thrown:
    - Only Firefox; both headless + Xvfb
    - OAC + RP
    - Both on HTTP and HTTPS
    - Response_ids: 185, 186, 341, 342; these are the ones that have status code 302!
- Reason seems to be that Firefox switches to another tab in these cases and thus the wait for crashes. Solution: forcefully switch back to the original tab before waiting!

In [40]:
with open("../crawler/log.log-main.log", "r") as f:
    crashes = []
    for l in f.readlines():
        if "main: http" in l:
            match = re.match(".*(http|https)://.*browser_id=(\d+)&label=(.*)&first_id=(\d+).*", l)
            crashes.append([match[1], match[2], match[3], match[4]])

In [41]:
crashes = pd.DataFrame(crashes, columns=["scheme", "browser_id", "label", "response_id"]).drop_duplicates()
crashes

,scheme,browser_id,label,response_id
0,http,14,OAC,342
2,http,14,RP,185
4,http,14,RP,186
6,https,14,OAC,341
8,https,14,OAC,342
10,https,14,RP,185
12,https,14,RP,186
14,http,72,OAC,341
16,http,72,OAC,342
18,http,72,RP,185
